In [33]:
from onnxruntime import InferenceSession
import polars as pl
import numpy as np
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from hydra import compose, initialize
from onnx import load_model

In [34]:
with initialize(version_base="1.3.2",
                config_path="../src/config",
                job_name="explainer_dashboard"):
    cfg = compose(config_name="config")

In [44]:
cfg.paths.data.X_test_transformed

'/Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/data/X_test_transformed.parquet'

In [52]:
X_test_transformed = pl.read_parquet(cfg.paths.data.X_test_transformed)

In [37]:
y_test = pl.read_parquet(cfg.paths.data.y_test)

In [53]:
X_test_transformed.glimpse()

Rows: 22929
Columns: 33
$ num__pre_icu_los_days                           <f64> -0.19279126437083538, -0.31054458355832015, 1.4576158653736913, -0.30283612695776885, -0.21405597228722034, -0.29778575903814797, -0.3153291428012303, -0.31772142242268536, -0.3163923782736025, -0.26163575565684644
$ cat__gender_F                                   <f64> 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0
$ cat__gender_M                                   <f64> 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0
$ cat__gender_None                                <f64> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
$ cat__ethnicity_African American                 <f64> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
$ cat__ethnicity_Asian                            <f64> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
$ cat__ethnicity_Caucasian                        <f64> 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0
$ cat__ethnicity_Hispanic                         <f64> 0.0, 0.0, 0.0, 0.0,

In [55]:
with open(cfg.paths.models.lr, "rb") as f:
    loaded_onnx_model = load_model(f)

In [56]:
sess = InferenceSession(loaded_onnx_model.SerializeToString(), providers=["CPUExecutionProvider"])

In [57]:
pred_ort = sess.run(None, {"X": X_test_transformed.astype(np.float32)})[0]

AttributeError: 'DataFrame' object has no attribute 'astype'

In [48]:
pred_skl = loaded_onnx_model.predict(X_test_transformed.astype(np.float32))

AttributeError: 'ModelProto' object has no attribute 'predict'

In [50]:
explainer = ClassifierExplainer(loaded_onnx_model, X_test_transformed, y_test.to_pandas(), shap='linear')

AssertionError: for ClassifierExplainer, model should be a scikit-learn compatible *classifier* model that has a predict_proba(...) method, so not a <class 'onnx.onnx_ml_pb2.ModelProto'>! If you are using e.g an SVM with hinge loss (which does not support predict_proba), you can try the following monkey patch:

import types
def predict_proba(self, X):
    pred = self.predict(X)
    return np.array([1-pred, pred]).T
model.predict_proba = types.MethodType(predict_proba, model)


In [ ]:
ExplainerDashboard(explainer).run()